# Pixel Intensity

This notebook demonstrates how to calculate the respiratory rate from the average pixel intensity in the chest region of a video.

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

subject = 'Proband16'
scenario = '101_natural_lighting'

num_frames = 30 * 20  # 20 seconds of video
frames, video_params = dataset.get_video_gray(subject, scenario, num_frames=num_frames)

## Calculate the average pixel intensity in the chest region

In [ ]:
import respiration.roi as roi
import respiration.extractor.pixel_intensity as pixel_intensity

chest_roi = roi.detect_chest(frames[0])

# Calculate the average pixel intensity in the chest region
average_pixels = pixel_intensity.average_pixel_intensity(frames, chest_roi)

In [ ]:
import matplotlib.pyplot as plt

# Plot the average pixel intensity in the chest region with matplotlib
plt.figure(figsize=(20, 5))
plt.plot(average_pixels)
plt.title('Average pixel intensity in the chest region')
plt.xlabel('Frame')
plt.ylabel('Average pixel intensity')
plt.show()

In [ ]:
ground_truth = dataset.get_breathing_signal(subject, scenario)

# Plot the average pixel intensity and the ground truth breathing signal in two subplots
fig, axs = plt.subplots(2, 1, figsize=(20, 10))

axs[0].plot(average_pixels)
axs[0].set_title('Average pixel intensity in the chest region')
axs[0].set_xlabel('Frame')
axs[0].set_ylabel('Average pixel intensity')

axs[1].plot(ground_truth[:len(average_pixels)])
axs[1].set_title('Ground truth breathing signal')
axs[1].set_xlabel('Frame')
axs[1].set_ylabel('Breathing signal')
plt.show()